In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class ST:
  def __init__(self, L, default, f):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, i):
    self[i] = self._def

  def __getitem__(self, i):
    return self.L[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.L[i] = v
    i >>= 1
    while i:
      self.L[i] = self._f(self.L[2 * i], self.L[2 * i + 1])
      i >>= 1

  def __len__(self):
    return self._len
  
  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

def sol():
  N = int(input())
  MAX = 200_000 * 2 #구분 가능한 y좌표는 40만개까지 나올 수 있다
  L = []
  comp = set() # 압축 -> 좌표 
  comp2 = {} # 좌표 -> 압축
  for _ in range(N):
    x1, x2, y1, y2 = map(int, input().split())
    comp.add(y1)
    comp.add(y2)
    L.append([x1, y1, y2, False])
    L.append([x2, y1, y2, True])

  L.sort()
  comp = sorted([*comp])
  for i, v in enumerate(comp):
    comp2[v] = i
  comp.append(comp[-1]) #좌표압축 계산을 위한

  for l in L :
    l[1] = comp2[l[1]]
    l[2] = comp2[l[2]]
  
  answer = 0
  st = ST([0] * MAX, 0, lambda x, y: x + y)
  st2 = ST([0] * MAX, 0, lambda x, y: x + y)
  last_x = L[0][0]
  for x, y1, y2, is_end in L:
    if x != last_x:
      dx = x - last_x
      assert(st.L[1] == st.query(0, MAX))
      answer += st2.query(0, MAX) * dx
      last_x = x

    for y in range(y1, y2) :
      st[y] += -1 if is_end else 1 #시작 수직선이면 +1, 끝이면 -1
      st2[y] = comp[y+1] - comp[y] if (st[y] > 0) else 0
      assert(st[y] >= 0), st[y]
  
  sys.stdout.write(str(answer) + '\n')

sol()

- TLE. 일단 의심되는건 좌표압축에 사용된 set이다. 이걸 다른방법으로 더 빠르게 할 수 있지 않을까?

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class ST:
  def __init__(self, L, default, f):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, i):
    self[i] = self._def

  def __getitem__(self, i):
    return self.L[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.L[i] = v
    i >>= 1
    while i:
      self.L[i] = self._f(self.L[2 * i], self.L[2 * i + 1])
      i >>= 1

  def __len__(self):
    return self._len
  
  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

def sol():
  N = int(input())
  MAX = 200_000 * 2 #구분 가능한 y좌표는 40만개까지 나올 수 있다
  L = []
  comp = set() # 압축 -> 좌표 
  comp2 = {} # 좌표 -> 압축
  for _ in range(N):
    x1, x2, y1, y2 = map(int, input().split())
    comp.add(y1)
    comp.add(y2)
    L.append([x1, y1, y2, False])
    L.append([x2, y1, y2, True])

  L.sort()
  comp = sorted([*comp])
  for i, v in enumerate(comp):
    comp2[v] = i
  comp.append(comp[-1]) #좌표압축 계산을 위한 더미값
  
  answer = 0
  st = ST([0] * MAX, 0, lambda x, y: x + y)
  st2 = [False] * MAX
  dy = 0
  last_x = L[0][0]
  for x, y1, y2, is_end in L:
    if x != last_x:
      dx = x - last_x
      assert(st.L[1] == st.query(0, MAX))
      answer += dy * dx
      last_x = x

    for y in range(comp2[y1], comp2[y2]) :
      st[y] += -1 if is_end else 1 #시작 수직선이면 +1, 끝이면 -1
      c = comp[y+1] - comp[y] if (st[y] > 0) else 0
      if st2[y] != c:
        dy -= st2[y] - c
        st2[y] = c
      assert(st[y] >= 0), st[y]
  
  sys.stdout.write(str(answer) + '\n')

sol()

- st2에서 세그먼트 트리를 없앤 구현으로 바꾸고, 좌표압축 부분을 조금 바꿔서 로컬 실행시간을 절반으로 줄였지만 여전히 TLE가 난다.
- 역시 구간쿼리를 처리하는데에 문제가 있는 것 같다. lazy propagation을 사용하지 않고 구현할 방법은 없을까? 

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class ST:
  def __init__(self, L, default, f):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, i):
    self[i] = self._def

  def __getitem__(self, i):
    return self.L[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.L[i] = v
    i >>= 1
    while i:
      self.L[i] = self._f(self.L[2 * i], self.L[2 * i + 1])
      i >>= 1

  def __len__(self):
    return self._len
  
  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

def sol():
  N = int(input())
  MAX = 200_000 * 2 #구분 가능한 y좌표는 40만개까지 나올 수 있다
  L = []
  comp = set() # 압축 -> 좌표 
  comp2 = {} # 좌표 -> 압축
  for _ in range(N):
    x1, x2, y1, y2 = map(int, input().split())
    comp.add(y1)
    comp.add(y2)
    L.append([x1, y1, y2, False])
    L.append([x2, y1, y2, True])

  L.sort()
  comp = sorted([*comp])
  for i, v in enumerate(comp):
    comp2[v] = i
  comp.append(comp[-1]) #좌표압축 계산을 위한 더미값
  
  answer = 0
  st = [0] * MAX
  st2 = [0] * MAX
  dy = 0
  last_x = L[0][0]
  for x, y1, y2, is_end in L:
    if x != last_x:
      dx = x - last_x
      answer += dy * dx
      last_x = x

    for y in range(comp2[y1], comp2[y2]) :
      st[y] += -1 if is_end else 1 #시작 수직선이면 +1, 끝이면 -1
      d = comp[y+1] - comp[y] if (st[y] > 0) else 0
      if st2[y] != d:
        dy -= st2[y] - d
        st2[y] = d
      assert(st[y] >= 0), st[y]
  
  sys.stdout.write(str(answer) + '\n')

sol()

;; 5
2
0 3 1 2
1 2 0 3
;; 5
2
5 8 6 7
6 7 5 8
;; 225
2
10 20 10 20
15 25 15 30

In [ ]:
import io, os, sys, bisect
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline
def sol():
  N = int(input())
  MAX = 200_000 * 2 #구분 가능한 y좌표는 40만개까지 나올 수 있다
  L = [(1,1,1,None)] * (N * 2)
  tmp = [] # 압축 -> 좌표 
  comp = []
  for i in range(0, N*2, 2):
    x1, x2, y1, y2 = map(int, input().split())
    tmp.append(y1)
    tmp.append(y2)
    L[i] = (x1, y1, y2, False)
    L[i+1] = (x2, y1, y2, True)

  L.sort(key=lambda x: x[0])
  tmp.sort()
  for i, v in enumerate(tmp):
    if v != tmp[i - 1]:
      comp.append(v)
  comp.append(comp[-1]) #좌표압축 계산을 위한 더미값
  
  answer = 0
  st = [0] * MAX
  st2 = [0] * MAX
  dy = 0
  last_x = L[0][0]
  for x, y1, y2, is_end in L:
    if x != last_x:
      dx = x - last_x
      answer += dy * dx
      last_x = x

    for y in range(bisect.bisect_left(comp, y1), bisect.bisect_left(comp, y2)) :
      st[y] += -1 if is_end else 1 #시작 수직선이면 +1, 끝이면 -1
      d = comp[y+1] - comp[y] if (st[y] > 0) else 0
      if st2[y] != d:
        dy -= st2[y] - d
        st2[y] = d
  
  sys.stdout.write(str(answer) + '\n')

sol()

- 해시 사용도 안했는데 TLE가 난다. 정렬과정에서 시간이 오래 걸린다고밖에 생각되지 않는다.
- 정렬에 문제가 있는걸까..?